please use python 3

In [1]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import UniSkip
from config import *
from datetime import datetime, timedelta
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs


In [2]:
d = DataLoader('../dir_HugeFiles/instructions/skip_instruction.csv')

Loading text file at ../dir_HugeFiles/instructions/skip_instruction.csv
Making dictionary for these words
Using cached dictionary at ../dir_HugeFiles/instructions/skip_instruction.csv.pkl
Making reverse dictionary
time: 330 ms


In [3]:
mod = UniSkip()
if USE_CUDA:
    mod.cuda(CUDA_DEVICE)

time: 6.8 s


In [4]:
lr = 3e-4
optimizer = torch.optim.Adam(params=mod.parameters(), lr=lr)

time: 19.2 ms


In [5]:
loss_trail = []
last_best_loss = None
current_time = datetime.utcnow()

def debug(i, loss, prev, nex, prev_pred, next_pred):
    global loss_trail
    global last_best_loss
    global current_time

    this_loss = loss.item()
    loss_trail.append(this_loss)
    loss_trail = loss_trail[-20:]
    new_current_time = datetime.utcnow()
    time_elapsed = str(new_current_time - current_time)
    current_time = new_current_time
    print("Iteration {}: time = {} last_best_loss = {}, this_loss = {}".format(
              i, time_elapsed, last_best_loss, this_loss))
    
    '''
    print("prev = {}\nnext = {}\npred_prev = {}\npred_next = {}".format(
        d.convert_indices_to_sentences(prev),
        d.convert_indices_to_sentences(nex),
        d.convert_indices_to_sentences(prev_pred),
        d.convert_indices_to_sentences(next_pred),
    ))
    '''

    try:
        trail_loss = sum(loss_trail)/len(loss_trail)
        if last_best_loss is None or last_best_loss > trail_loss:
            print("Loss improved from {} to {}".format(last_best_loss, trail_loss))
            
            save_loc = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
            print("saving model at {}".format(save_loc))
            torch.save(mod.state_dict(), save_loc)
            
            last_best_loss = trail_loss
    except Exception as e:
        print("Couldn't save model because {}".format(e))

time: 30.9 ms


In [6]:
print("Starting training...")
print('current GPU is on %d ' %(CUDA_DEVICE)) # should change at the config.py
# a million iterations
for i in range(0, 1000000):
    sentences, lengths = d.fetch_batch(32 * 8) # remember to change cuda device

    loss, prev, nex, prev_pred, next_pred  = mod(sentences, lengths)
    if i % 100 == 0:
        debug(i, loss, prev, nex, prev_pred, next_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Starting training...
current GPU is on 3 


/root/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration 0: time = 0:00:00.456559 last_best_loss = None, this_loss = 19.825910568237305
Loss improved from None to 19.825910568237305
saving model at ./saved_models/skip-best
Iteration 100: time = 0:01:17.574393 last_best_loss = 19.825910568237305, this_loss = 14.4572172164917
Loss improved from 19.825910568237305 to 17.141563892364502
saving model at ./saved_models/skip-best
Iteration 200: time = 0:01:16.605824 last_best_loss = 17.141563892364502, this_loss = 13.789891242980957
Loss improved from 17.141563892364502 to 16.02433967590332
saving model at ./saved_models/skip-best
Iteration 300: time = 0:01:16.859282 last_best_loss = 16.02433967590332, this_loss = 13.485515594482422
Loss improved from 16.02433967590332 to 15.389633655548096
saving model at ./saved_models/skip-best
Iteration 400: time = 0:01:16.547689 last_best_loss = 15.389633655548096, this_loss = 12.589254379272461
Loss improved from 15.389633655548096 to 14.829557800292969
saving model at ./saved_models/skip-best
Itera

KeyboardInterrupt: 

time: 14min 9s
